# Processing and Analysis of RNA-seq Data of Aging of Hematopoietic Stem Cells

Here I compare gene expression in hematopoietic stem cells extracted from 10-week-old mice ('young') vs 20-month-old mice ('aged').

[The data](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE162607) is from NCBI's Gene Expression Omnibus (GEO) repository. 

## download and process reference transcriptome and genome for Salmon

In [3]:
# download mouse reference transcriptome from gencode
!wget \
    -P ref_fasta \
    https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M32/gencode.vM32.transcripts.fa.gz

--2023-11-09 09:09:23--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M32/gencode.vM32.transcripts.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56246898 (54M) [application/x-gzip]
Saving to: ‘ref_fasta/gencode.vM32.transcripts.fa.gz’

100%[======================================>] 56,246,898   174KB/s   in 2m 49s 

2023-11-09 09:12:12 (326 KB/s) - ‘ref_fasta/gencode.vM32.transcripts.fa.gz’ saved [56246898/56246898]



In [4]:
# download mouse reference genome from gencode
!wget \
    -P ref_fasta \
    https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M32/GRCm39.primary_assembly.genome.fa.gz

--2023-11-09 09:13:48--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M32/GRCm39.primary_assembly.genome.fa.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 773873008 (738M) [application/x-gzip]
Saving to: ‘ref_fasta/GRCm39.primary_assembly.genome.fa.gz’

100%[======================================>] 773,873,008  276KB/s   in 40m 15s

2023-11-09 09:54:04 (313 KB/s) - ‘ref_fasta/GRCm39.primary_assembly.genome.fa.gz’ saved [773873008/773873008]



In [6]:
%%bash
# create decoy file
grep "^>" <(zcat ref_fasta/GRCm39.primary_assembly.genome.fa.gz) | cut -d " " -f 1 > ref_fasta/decoys.txt
sed -i -e 's/>//g' ref_fasta/decoys.txt

In [7]:
%%bash
# create gentrome
cat ref_fasta/gencode.vM32.transcripts.fa.gz ref_fasta/GRCm39.primary_assembly.genome.fa.gz > ref_fasta/gentrome.fa.gz

## quantification using Salmon of the 4 samples in NextFlow

In [2]:
!cat main.nf

#!/usr/bin/env nextflow

nextflow.enable.dsl=2

params.reads = "$baseDir/00_fastq_raw/*.fq"
params.outdir = "$baseDir/results"
params.gentrome = "$baseDir/ref_fasta/gentrome.fa.gz"
params.decoys = "$baseDir/ref_fasta/decoys.txt"

log.info """\
 R N A S E Q - N F   P I P E L I N E
 gentrome     : ${params.gentrome}
 reads        : ${params.reads}
 outdir       : ${params.outdir}
 """

process FASTQC {
    tag "FASTQC on $sample_id"
    conda 'bioconda::fastqc=0.12.1'
    publishDir "$params.outdir/01_fastq_raw_FastQC/", mode:'copy'

    cpus 2

    input:
    tuple val(sample_id), path(reads)

    output:
    path "fastqc_${sample_id}" 

    script:
    """
    mkdir -p fastqc_${sample_id}
    fastqc \\
        -t ${task.cpus} \\
        -o fastqc_${sample_id} \\
        ${reads}
    """
}

process INDEX {
    tag "$gentrome.simpleName"
    conda 'bioconda::salmon=1.10.2'

    cpus 28

    input:
    path gentrome 
    path decoys

    output:
    path 'index' 

    script:
    """
    

In [3]:
!cat nextflow.config

process.executor = 'slurm'
conda.enabled = true
conda.channels = 'seqera,conda-forge,bioconda,defaults'
conda.cacheDir = '/home/fmbuga/nf-conda-envs'

In [4]:
%%time
!nextflow run -resume \
    main.nf \
    -ansi-log false \
    -with-timeline \
    --reads "/home/fmbuga/scATAC/data/Itokawa22HSC/HSC_RNAseq/00_fastq_raw/*" 

N E X T F L O W  ~  version 23.10.0
Launching `main.nf` [scruffy_mcclintock] DSL2 - revision: b4605da6b0
 R N A S E Q - N F   P I P E L I N E
 gentrome     : /home/fmbuga/scATAC/data/Itokawa22HSC/HSC_RNAseq/test/ref_fasta/gentrome.fa.gz
 reads        : /home/fmbuga/scATAC/data/Itokawa22HSC/HSC_RNAseq/00_fastq_raw/*
 outdir       : /home/fmbuga/scATAC/data/Itokawa22HSC/HSC_RNAseq/test/results
 
[c8/1b7703] Submitted process > FASTQC (FASTQC on SRR13192301)
[b8/30d755] Submitted process > FASTQC (FASTQC on SRR13192285)
[89/fe041a] Submitted process > FASTQC (FASTQC on SRR13192302)
[78/9d54cd] Submitted process > TRIMGALORE (TRIMGALORE on SRR13192285)
[3c/26d95c] Submitted process > INDEX (gentrome)
[50/34dfe3] Submitted process > TRIMGALORE (TRIMGALORE on SRR13192301)
[08/3acb52] Submitted process > TRIMGALORE (TRIMGALORE on SRR13192302)
[f2/48421c] Submitted process > TRIMGALORE (TRIMGALORE on SRR13192286)
[cb/949e33] Submitted process > FASTQC (FASTQC on SRR13192286)
[26/fdecb2] Submit

In [14]:
# Import libraries for HTML visualization of timeline
from IPython.display import IFrame
IFrame(src='timeline-20231212-1861773.html', width=800, height=600)

In [5]:
ls results/03_salmon_quant

quant_SRR13192285/  quant_SRR13192286/  quant_SRR13192301/  quant_SRR13192302/


In [6]:
!cat results/03_salmon_quant/quant_SRR13192285/quant.sf | head

Name	Length	EffectiveLength	TPM	NumReads
ENSMUST00000193812.2	1070	819.000	0.000000	0.000
ENSMUST00000082908.3	110	3.000	0.000000	0.000
ENSMUST00000162897.2	4153	3902.000	0.000000	0.000
ENSMUST00000159265.2	2989	2738.000	0.000000	0.000
ENSMUST00000070533.5	3634	3383.000	0.000000	0.000
ENSMUST00000192857.2	480	229.000	0.000000	0.000
ENSMUST00000195335.2	2819	2568.000	0.000000	0.000
ENSMUST00000192336.2	2233	1982.000	0.000000	0.000
ENSMUST00000194099.2	2309	2058.000	0.000000	0.000
cat: write error: Broken pipe


In [7]:
import pandas as pd

pd.read_csv('results/03_salmon_quant/quant_SRR13192285/quant.sf', sep='\t').head()

,Name,Length,EffectiveLength,TPM,NumReads
0,ENSMUST00000193812.2,1070,819.0,0.0,0.0
1,ENSMUST00000082908.3,110,3.0,0.0,0.0
2,ENSMUST00000162897.2,4153,3902.0,0.0,0.0
3,ENSMUST00000159265.2,2989,2738.0,0.0,0.0
4,ENSMUST00000070533.5,3634,3383.0,0.0,0.0


In [10]:
!multiqc \
    --outdir results/04_multiqc \
    --verbose \
    work/


  /// ]8;id=758447;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.18

[2023-12-13 05:45:42] multiqc                                            [DEBUG  ]  This is MultiQC v1.18
[2023-12-13 05:45:42] multiqc                                            [DEBUG  ]  Command used: /home/fmbuga/.conda/envs/wxs-pipeline/bin/multiqc --outdir results/04_multiqc --verbose work/
[2023-12-13 05:45:42] multiqc                                            [DEBUG  ]  Latest MultiQC version is v1.17
[2023-12-13 05:45:42] multiqc                                            [DEBUG  ]  Working dir : /home/fmbuga/scATAC/data/Itokawa22HSC/HSC_RNAseq/test
[2023-12-13 05:45:42] multiqc                                            [DEBUG  ]  Template    : default
[2023-12-13 05:45:42] multiqc                                            [DEBUG  ]  Running Python 3.9.18 | packaged by conda-forge | (main, Aug 30 2023, 03:49:32)  [GCC 12.3.0]
[2023-12-13 05:45:42] multiqc                                            [DEBUG  

In [15]:
# Import libraries for HTML visualization of multiqc report
from IPython.display import IFrame
multiqc_report = "/home/fmbuga/scATAC/data/Itokawa22HSC/HSC_RNAseq/test/results/04_multiqc/multiqc_report.html"
IFrame(src="results/04_multiqc/multiqc_report.html", width=1200, height=900)

---

## NEXT: 
- R/Bioconductor analysis of quant.sf files